# H3-Toolkit Demo Data Generation & Visualization

This notebook demonstrates how to generate the 5 visualization boxes used in the `h3-toolkit` interactive demo and visualizes them interactively using Folium.

## 1. Setup Parameters
We use a **Resolution 7** origin cell (San Francisco) and trace its boundary at **Resolution 10**.

In [ ]:
import h3_toolkit
import h3
import json
import folium

# Center of the map (San Francisco)
lat = 37.773
lng = -122.418

# Parameters
origin_res = 7          # The large parent cell
intermediate_res = 10   # The fine resolution for boundary tracing

# Helper function to display maps
def show_map(geojson_data, color, title, opacity=0.4, fill=True):
    m = folium.Map(location=[lat, lng], zoom_start=13, tiles='CartoDB positron')
    
    # Add title as a popup or control (simplification: just rendering map)
    style_function = lambda x: {
        'fillColor': color,
        'color': color,
        'weight': 1 if fill else 2,
        'fillOpacity': opacity if fill else 0
    }
    
    folium.GeoJson(
        geojson_data,
        style_function=style_function,
        name=title
    ).add_to(m)
    
    return m

## 2. Get Origin Cell (Box 1)
The original large hexagon.
**Legend:** Green (Original Cell)

In [ ]:
origin = h3.latlng_to_cell(lat, lng, origin_res)
print(f"Origin Cell (Res {origin_res}): {origin}")

# Box 1: Original Cell Boundary
cell_boundary_gj = h3_toolkit.cell_boundary_to_geojson_cpp(origin)

# Visualize
show_map(cell_boundary_gj, '#238636', 'Original Cell', fill=False)

## 3. Get Boundary Children (Box 2)
Find all children at `intermediate_res` that lie on the 12 edges of the parent.
**Legend:** Orange (Boundary Children), Green (Original)

In [ ]:
boundary_children = h3_toolkit.children_on_boundary_faces(origin, intermediate_res)
print(f"Found {len(boundary_children)} boundary children at Res {intermediate_res}")

boundary_children_gj = {
    'type': 'FeatureCollection',
    'features': [h3_toolkit.cell_boundary_to_geojson_cpp(c) for c in boundary_children]
}

# Visualize (adding origin for context)
m2 = show_map(cell_boundary_gj, '#238636', 'Original Cell', fill=False)
folium.GeoJson(
    boundary_children_gj,
    style_function=lambda x: {'color': '#f0883e', 'weight': 1, 'fillOpacity': 0.6}
).add_to(m2)
m2

## 4. Merged Boundary (Box 3)
Merge the boundary children into a single polygon geometry.
**Legend:** Blue (Merged Boundary), Green (Original)

In [ ]:
merged_boundary_gj = h3_toolkit.cell_boundary_from_children_cpp(origin, intermediate_res)

# Visualize
m3 = show_map(cell_boundary_gj, '#238636', 'Original Cell', fill=False)
folium.GeoJson(
    merged_boundary_gj,
    style_function=lambda x: {'color': '#1f6feb', 'weight': 2, 'fillOpacity': 0.4}
).add_to(m3)
m3

## 5. Buffered Polygons (Box 4 & 5)
Generate polygons guaranteed to contain all child cells.
- **Accurate**: Uses the merged boundary.
- **Fast**: Uses a convex hull approximation.
- **Buffer**: `None` triggers auto-calculation (100% of edge length).

In [ ]:
# Box 4: Accurate Buffer (No Convex Hull)
buffered_accurate_gj = h3_toolkit.get_buffered_boundary_polygon_cpp(
    origin, 
    intermediate_res, 
    buffer_meters=None, 
    use_convex_hull=False
)

# Visualize Accurate Buffer (Red)
m4 = show_map(cell_boundary_gj, '#238636', 'Original Cell', fill=False)
folium.GeoJson(
    buffered_accurate_gj,
    style_function=lambda x: {'color': '#f85149', 'weight': 2, 'fillOpacity': 0.4}
).add_to(m4)
m4

In [ ]:
# Box 5: Fast Buffer (With Convex Hull)
buffered_fast_gj = h3_toolkit.get_buffered_boundary_polygon_cpp(
    origin, 
    intermediate_res, 
    buffer_meters=None, 
    use_convex_hull=True
)

# Visualize Comparison (Fast=Purple, Accurate=Red Ghost)
m5 = show_map(cell_boundary_gj, '#238636', 'Original Cell', fill=False)

# Add accurate for reference (ghosted)
folium.GeoJson(
    buffered_accurate_gj,
    style_function=lambda x: {'color': '#f85149', 'weight': 1, 'fillOpacity': 0.1, 'dashArray': '5, 5'}
).add_to(m5)

# Add fast buffer
folium.GeoJson(
    buffered_fast_gj,
    style_function=lambda x: {'color': '#a371f7', 'weight': 2, 'fillOpacity': 0.4}
).add_to(m5)
m5

## 6. Output to File
Save variables to a JS file for the demo page.

In [ ]:
output_file = 'demo_data.js'
print(f"Writing to {output_file}...")
with open(output_file, 'w') as f:
    f.write('var cellBoundary = ' + json.dumps(cell_boundary_gj) + ';\n')
    f.write('var boundaryChildren = ' + json.dumps(boundary_children_gj) + ';\n')
    f.write('var mergedBoundary = ' + json.dumps(merged_boundary_gj) + ';\n')
    f.write('var bufferedAccurate = ' + json.dumps(buffered_accurate_gj) + ';\n')
    f.write('var bufferedFast = ' + json.dumps(buffered_fast_gj) + ';\n')
print("Done.")